In [64]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.models import vgg16, VGG16_Weights


In [65]:
#set numpy print options
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
importance_scores = np.load('Ranking_arrays/importance_scores.npy')
#sort the importance scores
sorted_by_index_scores = importance_scores[importance_scores[:, 0].argsort()][:,1]

In [66]:
class QualityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg16(weights=VGG16_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # Predict quality and realness
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features

In [67]:
# Load pruned models 
BASELINE_MODEL_PATH = 'Weights/VGG-16_finetuned_regression.pth'
NOISY_PRUNED_MODEL_PATH = 'Weights/noise_out_pruned_model.pth'
NEGATIVE_IMPACT_PRUNED_MODEL_PATH = 'Weights/negative_impact_pruned_model.pth'

noisy_pruned_model = QualityPredictor()
noisy_pruned_model.load_state_dict(torch.load(NOISY_PRUNED_MODEL_PATH, weights_only=True))

negative_impact_pruned_model = QualityPredictor()
negative_impact_pruned_model.load_state_dict(torch.load(NEGATIVE_IMPACT_PRUNED_MODEL_PATH,weights_only=True))


def get_zeroed_feature_maps_indices(model, layer_name):
    """
    Get the indices of the zeroed out feature maps in a convolutional layer.

    Args:
        model (nn.Module): The neural network model.
        layer_name (str): The name of the convolutional layer.

    Returns:
        list: The indices of the zeroed out feature maps.
    """
    
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    zeroed_feature_maps = []

    for i, weight in enumerate(layer.weight):
        if torch.all(weight == 0):
            zeroed_feature_maps.append(i)
    zeroed_feature_maps.sort()

    return zeroed_feature_maps


In [ ]:
noisy_indices = get_zeroed_feature_maps_indices(noisy_pruned_model, 'features.28')
negative_indices = get_zeroed_feature_maps_indices(negative_impact_pruned_model, 'features.28')

print('Noisy Pruned Model:')
print(noisy_indices)

print('Negative Impact Pruned Model:')
print(negative_indices)



def z_score(score, mean, std):
    """
    Compute the Z-scores of a single score with respect to a set of scores."
    """
    z_score = (score - mean) / std
    return z_score

scores_mean = np.mean(sorted_by_index_scores)
scores_std = np.std(sorted_by_index_scores)

# Now i want to create an array of the same length as the importance scores array, where each element in the noisy_indices array is replaced with the z-score of the corresponding element in the importance scores array and the other elements are substituted with 1.



Noisy Pruned Model:
[1, 2, 3, 13, 14, 16, 24, 28, 32, 34, 40, 47, 49, 51, 60, 62, 67, 73, 76, 83, 85, 86, 87, 92, 93, 96, 99, 111, 113, 121, 123, 124, 125, 130, 136, 137, 138, 142, 147, 150, 153, 158, 162, 163, 165, 175, 182, 183, 184, 191, 193, 196, 200, 204, 212, 218, 220, 224, 233, 234, 236, 237, 239, 245, 250, 251, 254, 261, 262, 263, 264, 265, 276, 278, 281, 283, 288, 289, 290, 292, 293, 294, 299, 300, 302, 303, 320, 323, 325, 326, 330, 353, 356, 364, 375, 377, 381, 384, 387, 390, 391, 393, 401, 404, 407, 417, 422, 423, 425, 430, 437, 438, 441, 442, 447, 448, 451, 453, 454, 458, 461, 467, 472, 473, 476, 481, 482, 485, 493, 497, 499, 506, 510]
Negative Impact Pruned Model:
[1, 3, 4, 5, 6, 9, 13, 14, 19, 24, 26, 29, 30, 32, 34, 39, 40, 43, 45, 47, 49, 51, 55, 59, 62, 69, 73, 74, 76, 79, 80, 83, 86, 92, 93, 94, 95, 96, 99, 110, 112, 113, 115, 121, 129, 136, 137, 142, 144, 146, 147, 150, 152, 163, 164, 165, 168, 174, 175, 176, 177, 180, 183, 184, 190, 196, 198, 202, 205, 207, 209, 211